In [2]:
import os
import time
import subprocess
from datetime import datetime, timedelta
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# === CONFIGURACIÓN ===
#WATCH_FOLDER = r"C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\Test"
#OUTPUT_FOLDER = r"C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\Test"
#JAR_PATH = r"C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\boda-nokiacmdataparser.jar"
WATCH_FOLDER = r"C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML"
OUTPUT_FOLDER = r"C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML"
JAR_PATH = r"C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/boda-nokiacmdataparser.jar"
DIAS_PARA_LIMPIEZA = 7   # elimina dumps con más de 7 días
TIEMPO_ESPERA_COPIA = 10  # segundos entre verificaciones de tamaño

# === FUNCIONES ===

def archivo_completo(path):
    """Espera hasta que el archivo deje de crecer."""
    tamaño_anterior = -1
    while True:
        tamaño_actual = os.path.getsize(path)
        if tamaño_actual == tamaño_anterior:
            return True
        tamaño_anterior = tamaño_actual
        time.sleep(TIEMPO_ESPERA_COPIA)

def procesar_archivo(xml_path):
    nombre = os.path.basename(xml_path)
    print(f"[{datetime.now()}] Procesando: {nombre}")

    try:
        subprocess.run(
            ["java", "-jar", JAR_PATH, "-i", xml_path, "-o", OUTPUT_FOLDER],
            check=True
        )
        print(f"[{datetime.now()}] ✅ Conversión completada: {nombre}\n")
    except Exception as e:
        print(f"[{datetime.now()}] ❌ Error al procesar {nombre}: {e}\n")

def procesar_existentes():
    """Procesa archivos XML que ya existen al iniciar."""
    for file in os.listdir(WATCH_FOLDER):
        if file.endswith(".xml"):
            ruta = os.path.join(WATCH_FOLDER, file)
            print(f"[{datetime.now()}] Archivo existente encontrado: {file}")
            if archivo_completo(ruta):
                procesar_archivo(ruta)

def limpiar_archivos_viejos():
    """Elimina dumps viejos del folder."""
    ahora = datetime.now()
    for file in os.listdir(WATCH_FOLDER):
        if file.endswith(".xml"):
            ruta = os.path.join(WATCH_FOLDER, file)
            mod_time = datetime.fromtimestamp(os.path.getmtime(ruta))
            if ahora - mod_time > timedelta(days=DIAS_PARA_LIMPIEZA):
                os.remove(ruta)
                print(f"[{datetime.now()}] 🧹 Eliminado archivo antiguo: {file}")

# === EVENTO WATCHDOG ===
class DumpHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.endswith(".xml"):
            return

        archivo = event.src_path
        nombre = os.path.basename(archivo)
        print(f"[{datetime.now()}] Nuevo dump detectado: {nombre}")

        if archivo_completo(archivo):
            procesar_archivo(archivo)

# === LOOP PRINCIPAL ===
if __name__ == "__main__":
    os.makedirs(WATCH_FOLDER, exist_ok=True)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    print(f"=== HOT FOLDER NOKIA CM ACTIVO ===")
    print(f"Monitoreando: {WATCH_FOLDER}")
    print(f"Salida CSV:  {OUTPUT_FOLDER}")
    print(f"Limpieza automática: {DIAS_PARA_LIMPIEZA} días\n")

    # Procesar archivos existentes al iniciar
    procesar_existentes()

    # Iniciar monitor
    observer = Observer()
    observer.schedule(DumpHandler(), WATCH_FOLDER, recursive=False)
    observer.start()

    try:
        while True:
            observer.stop()
#            limpiar_archivos_viejos()
#            time.sleep(3600)  # limpieza cada hora
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


=== HOT FOLDER NOKIA CM ACTIVO ===
Monitoreando: C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML
Salida CSV:  C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Nokia/XML
Limpieza automática: 7 días

[2026-01-28 18:33:53.977083] Archivo existente encontrado: All_Nokia_20260126.xml
[2026-01-28 18:34:03.977910] Procesando: All_Nokia_20260126.xml
[2026-01-28 18:34:03.984799] ❌ Error al procesar All_Nokia_20260126.xml: [WinError 2] El sistema no puede encontrar el archivo especificado

